In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import math
import random
from geopy import distance
import swmmio
import pyswmm
from pyswmm import Simulation,Subcatchments
import math
import pickle
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import Point, box
import tifffile
import rasterio
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
from rasterio.transform import from_origin
from rasterio.enums import Resampling
import os   

In [ ]:

x_shift = 391688#extent coordinates
y_shift = 5816723#extent coordinates

path_tif = "C:/Users/sylvi/Documents/Hyspec4EO/for_dsieo/r2d_bln1_swmm_e1_1y_0H.tif"#read any tif file from ground truth H, V, U

# Define the paths to SWMM model files
inp_file = 'C:/Users/sylvi/Documents/Hyspec4EO/for_dsieo/for_dsieo/Berlin_I.inp'

path = "C:/Users/sylvi/Documents/Hyspec4EO/for_dsieo/for_dsieo/runoff1/"   #path for txt runoff files
path_save = "C:/Users/sylvi/Documents/Hyspec4EO/for_dsieo/for_dsieo/output/" #path for saving the output files
#however, the output files are saved in the same folder. please manage the output files in the different folders before run the save_pt ipynb
#example: output/b/b_100m/b_100mm_runoff_swmm_1200.tif
data= tifffile.imread(path_tif)
map=rasterio.open(path_tif)
print(map.crs)
print(map.bounds)
extent1 = [map.bounds[0], map.bounds[1], map.bounds[2], map.bounds[3]]
print(np.where(data!='nan'))
plt.imshow(data)
plt.show()

In [ ]:


# Import the SWMM model
Bln_1_model=swmmio.Model(inp_file)
subs=Bln_1_model.subcatchments.dataframe
# Reset the index to add the Name of the subcatchments as a column to the dataframe
subs = subs.reset_index()

# convert the coordinates to a polygon coordination that can be used by geopandas
subs['geometry']=subs['coords'].apply(lambda coords: Polygon(coords))

# Create a GeoDataFrame with the specified CRS
subs = gpd.GeoDataFrame(subs, geometry='geometry', crs='EPSG:25833')

# drop unnecessary data
subs.drop(columns=['Raingage', 'Outlet', 'PercImperv', 'Width',
       'PercSlope', 'CurbLength', 'N-Imperv', 'N-Perv', 'S-Imperv', 'S-Perv',
       'PctZero', 'RouteTo', 'PctRouted', 
        'coords', 'Area'],inplace=True)


# Create a new column with the center points of the polygons
subs['center_point'] = subs['geometry'].centroid
# Plot the GeoDataFrame
ax = subs.plot()

# Plot the center points on the same plot
subs['center_point'].plot(ax=ax, color='red', markersize=10)

extent2 = subs.total_bounds
extent = [min(extent1[0],extent2[0]), min(extent1[1],extent2[1]), max(extent1[2],extent2[2]), max(extent1[3],extent2[3])]

In [ ]:
# Create a raster with the same subcatchment id as the subcatchment
from rasterio.features import rasterize

#convert from subs  to raster
# Create a raster with the same extent as the map
raster = np.zeros((data.shape[0], data.shape[1]))
#give the point id as polygon value
# Define transform for raster (mapping between pixel coordinates and spatial coordinates)
transform = rasterio.transform.from_origin(extent[0], extent[3], 1, 1)
print(len(subs))
shapes = ((geom,value) for geom, value in zip(subs['geometry'], range(1, len(subs)+1)))
raster = rasterize(shapes=shapes, out_shape=(data.shape[0], data.shape[1]), transform=transform, fill=0, all_touched=True, dtype='int32')
plt.imshow(raster)

print(np.max(raster))
print(np.min(raster))



In [ ]:
def sum_data_5min(data):#sum the data in 5 min
    t=len(data)
    timestep = 10
    output = np.zeros((math.ceil(t/timestep)))
    for i in range(0,math.ceil(t/timestep)):
        output[i] = np.sum(data[i*timestep:(i+1)*timestep])
    return output


def get_nearest_runoff_value(px,py,data_txt_sum,subs,extent):
    x = px+extent[0]
    y = py+extent[1]
    point = Point(x,y)
    points = subs['center_point']
    distances = points.distance(point)
    nearest_point_id = distances.idxmin()
    return data_txt_sum[nearest_point_id]

#save to tif with projection


def save_to_tif(extent,file_name,image):

    # Set the transform with the upper left corner and the pixel size
    transform = from_origin(extent[0], extent[3], 1, 1)
    save_new_map = image.transpose(2,0,1)
    # Set the metadata
    meta = {
    'driver': 'GTiff',
    'height': save_new_map.shape[1],
    'width': save_new_map.shape[2],
    'count': save_new_map.shape[0],
    'dtype': save_new_map.dtype,
    'crs': CRS.from_epsg(25833),
    'transform': transform,
    'compress': 'lzw',
    'nodata': np.nan
    }

    # Write the raster to disk
    with rasterio.open(file_name, 'w', **meta) as dst:
        dst.write(save_new_map, indexes=[1])
        dst.nodata =np.nan
        dst.crs = CRS.from_epsg(25833)
        dst.transform = transform
        dst.close()

    # Open the raster file
    with rasterio.open(file_name) as src:
        #print(src.profile)
        #print(src.crs)
        #print(src.transform)
        #print(src.bounds)
        #print(src.width)
        #print(src.height)
        #print(src.count)
        #print(src.nodata)
        data = src.read(1)
        #plt.imshow(data)
        #plt.show()

In [ ]:

for file in os.listdir(path):
    if file.endswith(".txt"):
        path_txt = os.path.join(path, file)
        data_txt = []
        data_txt_sum = []
        for point_id in subs.index:
            data_txt.append(np.loadtxt(path_txt, skiprows=1,usecols=point_id+1))
            data_txt_sum.append(sum_data_5min(data_txt[point_id]))
        data_txt = np.array(data_txt)
        data_txt_sum = np.array(data_txt_sum)
        print(data_txt_sum.shape)
        fig=plt.figure()
        ax = fig.add_subplot(111)
        ax.set_title('plots')
        ax.set_xlabel('Time')
        ax.set_ylabel('Runoff')
        for i in range(0,subs.shape[0]):
            ax.plot(data_txt_sum[i])
        width = extent[2] - extent[0]
        height = extent[3] - extent[1]
        print(width,height)
        new_map = np.zeros((math.ceil(height),math.ceil(width),data_txt_sum.shape[1]))
        for px in range(0,math.ceil(width)):
            for py in range(0,math.ceil(height)):
                if raster[math.ceil(height)-py-1,px-1] != 0:
                    new_map[math.ceil(height)-py-1,px-1,:] = data_txt_sum[int(raster[math.ceil(height)-py-1,px-1])-1]
        print(new_map.shape)
        #plt.imshow(new_map[:,:,0])
        #plt.show()
        #new_map[np.where(raster==0)] = np.nan
        file_name = path_save+file[:-4]+".tif"
        folder_name = file_name.split("_runoff_swmm")[0].split("/")[-1]
        os.makedirs(path_save+folder_name, exist_ok=True)
        for i in range(0,25):
            file_name = path_save+folder_name+"/"+file[:-4]+"_"+str(i*300)+".tif"
            save_to_tif(extent,file_name,new_map[:,:,i].reshape(math.ceil(height),math.ceil(width),1))
            #print(file_name)

        
